In [1]:
# !pip install mediapipe opencv-python
# !pip3 install tensorflowjs
# !pip install protobuf==3.20.*
# !pip uninstall mediapipe
# !pip install mlserver==1.0.1 grpcio==3.20.1

In [2]:
import cv2
import mediapipe as mp
import tensorflow as tf
import csv
import numpy as np
import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder
# import tensorflowjs as tfjs
# import tensorflowjs as tfjs

2023-05-30 20:32:15.123723: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-30 20:32:15.212847: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-30 20:32:15.213961: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-30 20:32:16.959098: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

In [4]:
# cap = cv2.VideoCapture(0)
# with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
#   while cap.isOpened():
#     success, image = cap.read()
#     if not success:
#       print("Unable to record")
#       continue
#     image.flags.writeable = False
#     image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB )
#     results = pose.process(image)

#     image.flags.writeable = True
#     image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

#     mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, landmark_drawing_spec = mp_drawing_styles.get_default_pose_landmarks_style())

#     cv2.imshow('MediaPipe Pose', cv2.flip(image,1))

#     if cv2.waitKey(5) & 0xFF == ord('q'):
#       break
# cap.release()
# cv2.destroyAllWindows()

In [5]:
# def get_center_point(landmarks, left_bodypart, right_bodypart):
#     left = tf.gather(landmarks, left_bodypart.value, axis=1)
#     right = tf.gather(landmarks, right_bodypart.value, axis=1)
#     center = left * 0.5 + right * 0.5
#     return center


# def get_pose_size(landmarks, torso_size_multiplier=2.5):
#     hips_center = get_center_point(landmarks, BodyPart.LEFT_HIP, 
#                                  BodyPart.RIGHT_HIP)

#   # Shoulders center
#     shoulders_center = get_center_point(landmarks, BodyPart.LEFT_SHOULDER,
#                                       BodyPart.RIGHT_SHOULDER)

#   # Torso size as the minimum body size
#     torso_size = tf.linalg.norm(shoulders_center - hips_center)

#   # Pose center
#     pose_center_new = get_center_point(landmarks, BodyPart.LEFT_HIP, 
#                                      BodyPart.RIGHT_HIP)
#     pose_center_new = tf.expand_dims(pose_center_new, axis=1)
#   # Broadcast the pose center to the same size as the landmark vector to
#   # perform substraction
#     pose_center_new = tf.broadcast_to(pose_center_new,
#                                     [tf.size(landmarks) // (17*2), 17, 2])

#   # Dist to pose center
#     d = tf.gather(landmarks - pose_center_new, 0, axis=0,
#                 name="dist_to_pose_center")
#   # Max dist to pose center
#     max_dist = tf.reduce_max(tf.linalg.norm(d, axis=0))

#   # Normalize scale
#     pose_size = tf.maximum(torso_size * torso_size_multiplier, max_dist)

#     return pose_size


# def normalize_pose_landmarks(landmarks):
#     pose_center = get_center_point(landmarks, BodyPart.LEFT_HIP, 
#                                  BodyPart.RIGHT_HIP)
#     pose_center = tf.expand_dims(pose_center, axis=1)
#   # Broadcast the pose center to the same size as the landmark vector to perform
#   # substraction
#     pose_center = tf.broadcast_to(pose_center, 
#                                 [tf.size(landmarks) // (17*2), 17, 2])
#     landmarks = landmarks - pose_center

#   # Scale the landmarks to a constant pose size
#     pose_size = get_pose_size(landmarks)
#     landmarks /= pose_size

#     return landmarks


# def landmarks_to_embedding(landmarks_and_scores):
#     reshaped_inputs = keras.layers.Reshape((33, 3))(landmarks_and_scores)

#   # Normalize landmarks 2D
#     landmarks = normalize_pose_landmarks(reshaped_inputs[:, :, :2])

#   # Flatten the normalized landmark coordinates into a vector
#     embedding = keras.layers.Flatten()(landmarks)

#     return embedding

In [6]:
# def get_center_point(landmarks, left_bodypart, right_bodypart):
#     left = tf.gather(landmarks, left_bodypart, 1)
#     right = tf.gather(landmarks, right_bodypart, 1)
#     center = tf.add(tf.mul(left, 0.5), tf.mul(right, 0.5))
#     return center
  

# def get_pose_size(landmarks, torso_size_multiplier = 2.5):
#     hips_center = get_center_point(
#       landmarks,
#       POINTS.LEFT_HIP,
#       POINTS.RIGHT_HIP
#     )
#     shoulders_center = get_center_point(
#       landmarks,
#       POINTS.LEFT_SHOULDER,
#       POINTS.RIGHT_SHOULDER
#     )
#     torso_size = tf.norm(tf.sub(shoulders_center, hips_center))
#     pose_center_new = get_center_point(
#       landmarks,
#       POINTS.LEFT_HIP,
#       POINTS.RIGHT_HIP
#     )
#     pose_center_new = tf.expandDims(pose_center_new, 1)

# #     // pose_center_new = tf.broadcastTo(pose_center_new, [1, 17, 2]);
# #     // return: shape(17,2)
#     d = tf.gather(tf.sub(landmarks, pose_center_new), 0, 0)
#     max_dist = tf.max(tf.norm(d, "euclidean", 0))

# #     // normalize scale
#     pose_size = tf.maximum(
#       tf.mul(torso_size, torso_size_multiplier),
#       max_dist
#     )
#     return pose_size

# def normalize_pose_landmarks(landmarks):
#     pose_center = get_center_point(
#       landmarks,
#       POINTS.LEFT_HIP,
#       POINTS.RIGHT_HIP
#     )
#     pose_center = tf.expandDims(pose_center, 1)
# #     // pose_center = tf.broadcastTo(pose_center, [1, 17, 2])
#     landmarks = tf.sub(landmarks, pose_center)

#     pose_size = get_pose_size(landmarks)
#     landmarks = tf.div(landmarks, pose_size)
#     return landmarks

# def landmarks_to_embedding(landmarks):
# #     // normalize landmarks 2D
#     landmarks = normalize_pose_landmarks(tf.expandDims(landmarks, 0))
#     embedding = tf.reshape(landmarks, [1, 99])
# #     // console.log(embedding.array().then((data) => console.log(data)));
#     return embedding
  

In [68]:
# # Run this to generate pose coordinate
# csv_out_path = 'pose_coordinate.csv'
# with open(csv_out_path, 'w') as csv_out_file:
#     csv_out_writer = csv.writer(csv_out_file, delimiter=',', quoting=csv.QUOTE_MINIMAL)
    
#     basepath = 'train'
#     for entry in os.listdir(basepath):
#         folder_path = os.path.join(basepath, entry)
#         for img in os.listdir(folder_path):
#             with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
#                 image_path = os.path.join(folder_path,img)
#                 image = cv2.imread(image_path)

#                 image.flags.writeable = False
#                 image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB )
#                 result = pose.process(image)
#                 pose_landmarks = result.pose_landmarks
#                 output_frame = image.copy()
                
#                 if pose_landmarks is not None:
#                     # Check the number of landmarks and take pose landmarks.
#                     assert len(pose_landmarks.landmark) == 33, 'Unexpected number of predicted pose landmarks: {}'.format(len(pose_landmarks.landmark))
#                     pose_landmarks = [[lmk.x, lmk.y, lmk.z] for lmk in pose_landmarks.landmark]

#                     # Map pose landmarks from [0, 1] range to absolute coordinates to get
#                     # correct aspect ratio.
#                     frame_height, frame_width = output_frame.shape[:2]
#                     pose_landmarks *= np.array([frame_width, frame_height, frame_width])

#                     # Write pose sample to CSV.
#                     pose_landmarks = np.around(pose_landmarks, 5).flatten().astype(str).tolist()
#                     csv_out_writer.writerow([img, entry] + pose_landmarks)



In [8]:
# def get_center_point(pose_landmarks):
#     left_hip = pose_landmarks.landmark[23]
#     right_hip = pose_landmarks.landmark[24]
#     hip_center_x = (left_hip.x + right_hip.x)*0.5;
#     hip_center_y = (left_hip.y + right_hip.y)*0.5;
                    
#     left_shoulder = pose_landmarks.landmark[11]
#     right_shoulder = pose_landmarks.landmark[12]
#     shoulder_center_x = (left_shoulder.x + right_shoulder.x)*0.5;
#     shoulder_center_y = (left_shoulder.y + right_shoulder.y)*0.5;
                    
#     torso_center_x = (hip_center_x + shoulder_center_x )*0.5;
#     torso_center_y = (hip_center_y + shoulder_center_y )*0.5;

#     return torso_center_x, torso_center_y

In [9]:
# def normalize(landmarks,pose_center_new):
#     d = tf.gather(landmarks - pose_center_new, 0, axis=0,
#                 name="dist_to_pose_center")
#       # Max dist to pose center
#     max_dist = tf.reduce_max(tf.linalg.norm(d, axis=0))

#       # Normalize scale
#     pose_size = tf.maximum(torso_size * torso_size_multiplier, max_dist)

In [10]:

# csv_out_path = 'pose_coordinate.csv'
# with open(csv_out_path, 'w') as csv_out_file:
#     csv_out_writer = csv.writer(csv_out_file, delimiter=',', quoting=csv.QUOTE_MINIMAL)
    
#     basepath = 'train'
#     for entry in os.listdir(basepath):
#         folder_path = os.path.join(basepath, entry)
#         for img in os.listdir(folder_path):
#             with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
#                 image_path = os.path.join(folder_path,img)
#                 image = cv2.imread(image_path)

#                 image.flags.writeable = False
#                 image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB )
#                 result = pose.process(image)
#                 pose_landmarks = result.pose_landmarks
#                 output_frame = image.copy()
                
#                 if pose_landmarks is not None:
#                     torso_center_x, torso_center_y = get_center_point(pose_landmarks)
#                     print(torso_center_x, torso_center_y)
# #                     # Check the number of landmarks and take pose landmarks.
#                     assert len(pose_landmarks.landmark) == 33, 'Unexpected number of predicted pose landmarks: {}'.format(len(pose_landmarks.landmark))
#                     pose_landmarks = [[lmk.x - torso_center_x, lmk.y - torso_center_y, lmk.z] for lmk in pose_landmarks.landmark]

#                     # Map pose landmarks from [0, 1] range to absolute coordinates to get
#                     # correct aspect ratio.
#                     frame_height, frame_width = output_frame.shape[:2]
#                     pose_landmarks *= np.array([frame_width, frame_height, frame_width])

#                     # Write pose sample to CSV.
#                     pose_landmarks = np.around(pose_landmarks, 5).flatten().astype(str).tolist()
#                     csv_out_writer.writerow([img, entry] + pose_landmarks)



In [11]:

# csv_out_path = 'pose_coordinate.csv'
# with open(csv_out_path, 'w') as csv_out_file:
#     csv_out_writer = csv.writer(csv_out_file, delimiter=',', quoting=csv.QUOTE_MINIMAL)
    
#     basepath = 'train'
#     for entry in os.listdir(basepath):
#         folder_path = os.path.join(basepath, entry)
#         for img in os.listdir(folder_path):
#             with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
#                 image_path = os.path.join(folder_path,img)
#                 image = cv2.imread(image_path)

#                 image.flags.writeable = False
#                 image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB )
#                 result = pose.process(image)
#                 pose_landmarks = result.pose_landmarks
#                 output_frame = image.copy()
                
#                 if pose_landmarks is not None:
# #                     torso_center_x, torso_center_y = get_center_point(pose_landmarks)
# #                     print(torso_center_x, torso_center_y)
# #                     # Check the number of landmarks and take pose landmarks.
#                     assert len(pose_landmarks.landmark) == 33, 'Unexpected number of predicted pose landmarks: {}'.format(len(pose_landmarks.landmark))
#                     pose_landmarks = [[lmk.x , lmk.y , lmk.z] for lmk in pose_landmarks.landmark]
#                     pose_landmarks = landmarks_to_embedding(pose_landmarks)
#                     # Map pose landmarks from [0, 1] range to absolute coordinates to get
#                     # correct aspect ratio.
#                     frame_height, frame_width = output_frame.shape[:2]
#                     pose_landmarks *= np.array([frame_width, frame_height, frame_width])

#                     # Write pose sample to CSV.
#                     pose_landmarks = np.around(pose_landmarks, 5).flatten().astype(str).tolist()
#                     csv_out_writer.writerow([img, entry] + pose_landmarks)



In [69]:
df = pd.read_csv('pose_coordinate.csv',names=[x for x in range(101)])

In [70]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,girl1_chair113.jpg,chair,169.22336,100.65133,-176.83078,172.41418,95.42514,-168.56890,174.40276,95.22239,...,-74.94151,169.35428,245.76121,-54.18222,170.86158,257.23218,-132.79347,162.85747,257.09365,-109.37694
1,girl2_chair105.jpg,chair,186.07258,94.28657,23.96739,183.95346,91.02365,19.29216,183.16904,91.32135,...,32.88076,179.51996,271.26281,-68.39649,185.97307,272.98824,18.63087,190.95989,275.44101,-85.40584
2,guy2_chair076_flipped.jpg,chair,171.74335,104.13264,-161.98461,171.19639,100.09763,-152.59674,171.72500,99.95775,...,61.71619,155.45043,279.93286,-15.13519,179.43227,286.79556,31.48036,176.41298,287.81515,-47.79021
3,girl3_chair132.jpg,chair,138.44002,155.35533,-250.67323,141.43345,153.48882,-255.23221,143.20740,153.85711,...,-32.27770,146.91974,215.91039,62.14866,146.23427,227.85920,-67.20390,140.70244,224.59740,30.94406
4,girl3_chair131_flipped.jpg,chair,158.95575,155.67246,-250.17040,160.13260,151.86371,-248.30956,161.28139,150.50583,...,65.58166,148.42978,221.29594,-23.33949,168.30679,220.62507,35.94082,158.83529,228.21847,-54.32943


In [71]:
df.shape

(2544, 101)

In [72]:
df.columns

Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
       ...
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100],
      dtype='int64', length=101)

In [73]:
df.drop(0,axis=1,inplace=True)

In [74]:
df.iloc[:,1:].values

array([[ 169.22336,  100.65133, -176.83078, ...,  162.85747,  257.09365,
        -109.37694],
       [ 186.07258,   94.28657,   23.96739, ...,  190.95989,  275.44101,
         -85.40584],
       [ 171.74335,  104.13264, -161.98461, ...,  176.41298,  287.81515,
         -47.79021],
       ...,
       [ 398.19064,  145.55058, -368.60132, ...,  381.54676,  727.15197,
          16.68096],
       [ 326.1998 ,   79.14401, -342.47997, ...,  333.83682,  433.58791,
         -15.88974],
       [ 413.20887,   80.69553, -341.38753, ...,  358.34675,  764.64844,
          41.53072]])

In [75]:
x = df.iloc[:,1:].values

In [76]:
lis = [x for x in df[1]]

In [77]:
le = LabelEncoder()

In [78]:
le.fit(df[1])

LabelEncoder()

In [79]:
lis = le.transform(df[1])

In [80]:
df[1].unique()

array(['chair', 'traingle', 'warrior Balanced', 'tree', 'shoudler_stand',
       'cobra', 'dog', 'warrior', 'no_pose'], dtype=object)

In [81]:
lis

array([0, 0, 0, ..., 3, 3, 3])

In [82]:
import keras

In [83]:
y = keras.utils.to_categorical(lis,num_classes=9)

In [84]:
y.shape

(2544, 9)

In [85]:
from sklearn.model_selection import train_test_split

In [86]:
x_train, x_test, y_train, y_test = train_test_split(x,y,random_state=42)

In [87]:
x_train.shape

(1908, 99)

In [88]:
from keras.layers import Dense, Dropout
from keras.models import Sequential

In [89]:
x.shape

(2544, 99)

In [90]:
model = Sequential()
model.add(Dense(256, activation='relu', input_dim=x.shape[1]))
model.add(Dropout(.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(.2))
model.add(Dense(64, activation='relu'))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 256)               25600     
                                                                 
 dropout_2 (Dropout)         (None, 256)               0         
                                                                 
 dense_5 (Dense)             (None, 128)               32896     
                                                                 
 dropout_3 (Dropout)         (None, 128)               0         
                                                                 
 dense_6 (Dense)             (None, 64)                8256      
                                                                 
 dense_7 (Dense)             (None, 9)                 585       
                                                                 
Total params: 67,337
Trainable params: 67,337
Non-trai

In [91]:
callback = keras.callbacks.EarlyStopping(monitor='loss', patience=20, min_delta =0.1 ,restore_best_weights=True)

In [92]:
history = model.fit(x_train, y_train, batch_size=25,epochs=1000,callbacks=[callback])

Epoch 1/1000
77/77 [==============================] - 1s 3ms/step - loss: 27.3883 - accuracy: 0.3422
Epoch 2/1000
77/77 [==============================] - 0s 3ms/step - loss: 6.3562 - accuracy: 0.4549
Epoch 3/1000
77/77 [==============================] - 0s 3ms/step - loss: 3.4136 - accuracy: 0.5246
Epoch 4/1000
77/77 [==============================] - 0s 3ms/step - loss: 2.3150 - accuracy: 0.5571
Epoch 5/1000
77/77 [==============================] - 0s 3ms/step - loss: 1.9451 - accuracy: 0.5839
Epoch 6/1000
77/77 [==============================] - 0s 3ms/step - loss: 1.6161 - accuracy: 0.6504
Epoch 7/1000
77/77 [==============================] - 0s 3ms/step - loss: 1.4905 - accuracy: 0.6410
Epoch 8/1000
77/77 [==============================] - 0s 3ms/step - loss: 1.2448 - accuracy: 0.6630
Epoch 9/1000
77/77 [==============================] - 0s 3ms/step - loss: 1.0899 - accuracy: 0.7254
Epoch 10/1000
77/77 [==============================] - 0s 3ms/step - loss: 0.9747 - accuracy: 0.738

In [93]:
model.save("model")

2023-05-30 20:54:17.982967: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,256]
	 [[{{node inputs}}]]
2023-05-30 20:54:18.021223: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,128]
	 [[{{node inputs}}]]
2023-05-30 20:54:18.469208: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,256]
	 [[{{node inputs}}]]
2023-05-30 20:54

INFO:tensorflow:Assets written to: model/assets


INFO:tensorflow:Assets written to: model/assets


In [94]:
model.predict(x_test)

20/20 [==============================] - 0s 2ms/step


array([[8.5243833e-01, 1.1508827e-05, 3.8731341e-06, ..., 1.1040691e-01,
        2.9043164e-02, 7.0936297e-04],
       [2.2442503e-16, 5.8252621e-09, 9.9999958e-01, ..., 1.3518476e-13,
        5.2700484e-09, 8.2881157e-10],
       [0.0000000e+00, 0.0000000e+00, 9.9999994e-01, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       ...,
       [1.3782707e-02, 3.8755694e-05, 3.3228632e-04, ..., 9.7968310e-01,
        4.4197887e-03, 1.6420784e-04],
       [6.4850549e-11, 3.0466589e-07, 9.9948043e-01, ..., 5.5994659e-10,
        1.5424424e-05, 5.0364912e-04],
       [2.4136261e-07, 3.6241770e-05, 9.8611706e-01, ..., 3.4262135e-07,
        1.8570045e-04, 1.3561534e-02]], dtype=float32)

In [95]:
model.evaluate(x_test,y_test)

20/20 [==============================] - 0s 3ms/step - loss: 0.2983 - accuracy: 0.9513


[0.2983179986476898, 0.9512578845024109]

In [96]:
model.get_weights()

[array([[ 0.06161332, -0.11456384,  0.10003474, ..., -0.06764895,
         -0.06690582, -0.12889852],
        [ 0.05018184,  0.11470892,  0.04976233, ..., -0.00503258,
         -0.0969696 ,  0.00695337],
        [-0.02210096,  0.10371587, -0.02606583, ..., -0.00318382,
          0.11826818,  0.00270424],
        ...,
        [ 0.02555225, -0.13829172,  0.12736723, ..., -0.07354182,
         -0.09597609,  0.01682347],
        [ 0.01986338,  0.08114799,  0.1072642 , ...,  0.04795874,
         -0.0682776 ,  0.08024067],
        [-0.11891891,  0.00138167,  0.02743996, ...,  0.06633289,
         -0.17398168, -0.04217749]], dtype=float32),
 array([-1.97972208e-02, -2.14659907e-02,  1.92883685e-02, -8.11673794e-03,
        -1.56329907e-02, -6.88633174e-02, -1.22049730e-02, -3.22576798e-02,
        -5.14228176e-03, -1.26561727e-02, -1.22407386e-02, -1.28109064e-02,
         7.20164832e-03, -1.47156678e-02,  8.89929309e-02,  7.26072714e-02,
        -8.82665068e-03, -5.82639426e-02, -1.18379489e

In [97]:
model.save("model")

2023-05-30 20:54:20.475814: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,256]
	 [[{{node inputs}}]]
2023-05-30 20:54:20.501633: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,128]
	 [[{{node inputs}}]]
2023-05-30 20:54:20.800471: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,256]
	 [[{{node inputs}}]]
2023-05-30 20:54

INFO:tensorflow:Assets written to: model/assets


INFO:tensorflow:Assets written to: model/assets


In [98]:
np.expand_dims(x_test[0],axis=0).shape

(1, 99)

In [99]:
import pickle

In [100]:
pickle.dump(model, open('model_pkl.pkl','wb'))

In [101]:
# model_pkl = pickle.load(open('model_pkl.pkl','rb'))

In [102]:
model.predict(np.expand_dims(x_test[0],axis=0))

1/1 [==============================] - 0s 29ms/step


array([[8.5243845e-01, 1.1508829e-05, 3.8731346e-06, 2.6731321e-04,
        3.3478200e-06, 7.1161534e-03, 1.1040677e-01, 2.9043168e-02,
        7.0936303e-04]], dtype=float32)

In [103]:
# mapping = dict(zip(le.classes_, le.transform(le.classes_)))

In [104]:
le.inverse_transform([0])

array(['chair'], dtype=object)

In [105]:
cap = cv2.VideoCapture(0)
with mp_pose.Pose(min_detection_confidence=0.3, min_tracking_confidence=0.3) as pose:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Unable to record")
            continue
        image.flags.writeable = False
        image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB )
        results = pose.process(image)
        pose_landmarks = results.pose_landmarks
        output_frame = image.copy()
        pose_name = 'Unidentified'
        if pose_landmarks is not None:
#             torso_center_x, torso_center_y = get_center_point(pose_landmarks)
            vector = [[lmk.x, lmk.y, lmk.z] for lmk in pose_landmarks.landmark]
            frame_height, frame_width = output_frame.shape[:2]

            vector *= np.array([frame_width, frame_height, frame_width])
            vector = vector.flatten()
            vector = np.expand_dims(vector,axis=0)

            prediction = model.predict((vector))

            index = np.argmax(prediction)

            pose_name = le.inverse_transform([index])
#             print("pose name:",pose_name)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        mp_drawing.draw_landmarks(image, pose_landmarks, mp_pose.POSE_CONNECTIONS, landmark_drawing_spec = mp_drawing_styles.get_default_pose_landmarks_style())

        image = cv2.flip(image,1)

        #Rectangle
        start_point = (0, 0)
        end_point = (300, 100)
        color = (0, 0, 0)
        thickness = -1
        image = cv2.rectangle(image, start_point, end_point, color, thickness)
        #write
        font = cv2.FONT_HERSHEY_SIMPLEX
        org = (25, 50)
        fontScale = 1
        color = (255, 255, 255)
        thickness = 2

        image = cv2.putText(image, pose_name[0], org, font, 
                       fontScale, color, thickness, cv2.LINE_AA)
        cv2.imshow('MediaPipe Pose',image)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 25ms/step


In [106]:
#EOF

In [107]:
df[1].unique()

array(['chair', 'traingle', 'warrior Balanced', 'tree', 'shoudler_stand',
       'cobra', 'dog', 'warrior', 'no_pose'], dtype=object)

In [108]:
pose_dict={};

In [109]:
for pose in df[1].unique():
    pose_dict[str(le.transform([pose])[0])] = pose

In [110]:
le.transform(['chair'])

array([0])

In [111]:
pose_dict

{'0': 'chair',
 '5': 'traingle',
 '8': 'warrior Balanced',
 '6': 'tree',
 '4': 'shoudler_stand',
 '1': 'cobra',
 '2': 'dog',
 '7': 'warrior',
 '3': 'no_pose'}

In [112]:
import json

In [113]:
with open('pose_labels.txt','w') as f:
    f.write(json.dumps(pose_dict))

In [114]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

print('Model size: %dKB' % (len(tflite_model) / 1024))

with open('pose_classifier.tflite', 'wb') as f:
  f.write(tflite_model)

2023-05-30 20:54:55.946060: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,256]
	 [[{{node inputs}}]]
2023-05-30 20:54:55.966014: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,128]
	 [[{{node inputs}}]]
2023-05-30 20:54:56.175523: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,256]
	 [[{{node inputs}}]]
2023-05-30 20:54

INFO:tensorflow:Assets written to: /tmp/tmp2bva7pll/assets


INFO:tensorflow:Assets written to: /tmp/tmp2bva7pll/assets


Model size: 71KB


2023-05-30 20:54:57.564703: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-05-30 20:54:57.564759: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-05-30 20:54:57.565142: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmp2bva7pll
2023-05-30 20:54:57.568030: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-05-30 20:54:57.568085: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmp2bva7pll
2023-05-30 20:54:57.580104: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2023-05-30 20:54:57.668570: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundle at path: /tmp/tmp2bva7pll
2023-05-30 20:54:57.694532: I tensorflow/cc/saved_model/loader.cc:314] SavedModel load for tags { serve }; Status: success: OK. Took 129392 

In [115]:
# with open('pose_labels.txt', 'w') as f:
#   f.write('\n'.join(df[1]))

In [116]:
# from zipfile import ZipFile

In [117]:
# with ZipFile('model.zip', 'w') as zipObj:
#     zipObj.write('pose_classifier.tflite')
#     zipObj.write('pose_labels.txt')
#     zipObj.close()

In [118]:
# tfjs.converters.save_keras_model(model,'model')

In [119]:
# !zip pose_classifier.zip pose_labels.txt pose_classifier.tflite

In [120]:
# try:
#   from google.colab import files
#   files.download('pose_classifier.zip')
# except:
#   pass

In [121]:
model.predict(x_test)

20/20 [==============================] - 0s 2ms/step


array([[8.5243833e-01, 1.1508827e-05, 3.8731341e-06, ..., 1.1040691e-01,
        2.9043164e-02, 7.0936297e-04],
       [2.2442503e-16, 5.8252621e-09, 9.9999958e-01, ..., 1.3518476e-13,
        5.2700484e-09, 8.2881157e-10],
       [0.0000000e+00, 0.0000000e+00, 9.9999994e-01, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       ...,
       [1.3782707e-02, 3.8755694e-05, 3.3228632e-04, ..., 9.7968310e-01,
        4.4197887e-03, 1.6420784e-04],
       [6.4850549e-11, 3.0466589e-07, 9.9948043e-01, ..., 5.5994659e-10,
        1.5424424e-05, 5.0364912e-04],
       [2.4136261e-07, 3.6241770e-05, 9.8611706e-01, ..., 3.4262135e-07,
        1.8570045e-04, 1.3561534e-02]], dtype=float32)

In [122]:
model.evaluate(x_test, y_test)

20/20 [==============================] - 0s 2ms/step - loss: 0.2983 - accuracy: 0.9513


[0.2983179986476898, 0.9512578845024109]